### Notation:
- SAL- small area
- PP- police precinct
- AEA- Albers Equal Area Conic

In [1]:
from random import shuffle, randint
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from mpl_toolkits.basemap import Basemap
from shapely.geometry import Polygon, Point, MultiPoint, MultiPolygon, LineString, mapping, shape
from descartes import PolygonPatch
import random
import fiona
import numpy as np
import csv
from fiona import collection

import geopandas as gpd
from geopandas.tools import sjoin # rtree index in-build, used with inner, intersection
import pandas as pd

from collections import defaultdict


def sjoin(left_df, right_df, how='inner', op='intersects',
          lsuffix='left', rsuffix='right', **kwargs):
    """Spatial join of two GeoDataFrames.
    left_df, right_df are GeoDataFrames
    how: type of join
        left -> use keys from left_df; retain only left_df geometry column
        right -> use keys from right_df; retain only right_df geometry column
        inner -> use intersection of keys from both dfs;
                 retain only left_df geometry column
    op: binary predicate {'intersects', 'contains', 'within'}
        see http://toblerity.org/shapely/manual.html#binary-predicates
    lsuffix: suffix to apply to overlapping column names (left GeoDataFrame)
    rsuffix: suffix to apply to overlapping column names (right GeoDataFrame)
    """


In [2]:
def find_intersections(o):
    
    from collections import defaultdict

    paired_ind = [o.pp_index, o.sal_index]

    d_over_ind = defaultdict(list)

    # creating a dictionary that has prescints as keys and associated small areas as values
    for i in range(len(paired_ind[0].values)):
        if not paired_ind[0].values[i]==paired_ind[1].values[i]: # it shows itself as intersection
            d_over_ind[paired_ind[0].values[i]].append(paired_ind[1].values[i])

    # get rid of the pol precincts with no small areas associated to them- not the most efficient way
    d_temp = {}
    for l in d_over_ind:
        if len(d_over_ind[l]):
            d_temp[l] = d_over_ind[l]

    return d_temp
    
    
def calculate_join_indices(g1_reind, g2_reind):

        # A: region of the police data with criminal record
        # C: small area with population data
        # we look for all small areas intersecting a given C_i, calculate the fraction of inclusion, scale the
        # population accordingly: area(A_j, where A_j crosses C_i)/area(A_j)* popul(A_j)
        
    
        # the actual indexing:
        out = sjoin(g1_reind, g2_reind, how ="inner", op = "intersects")
        
        out.drop('index_right', axis=1, inplace=True) # there is a double index fo smal areas, so we drop one
        #out_sorted = out.sort(columns='polPrecincts_index', ascending=True) # guess sorting is not necessary, cause we are
        # using doctionaries at later stages
        #dict_over_ind = find_intersections(out_sorted)

        # output retains only 1 area (left or right join), and gives no intersection area.
        # so we create an array with paired indices: police precincts with associated small areas
        # we use it in a loop in a function below
        dict_over_ind = find_intersections(out) 
        
        return dict_over_ind
    
def calculate_inclusion_indices(g1_reind, g2_reind):

        out = sjoin(g1_reind, g2_reind, op = "contains") ## PP contains SAL
        
        out.drop('index_right', axis=1, inplace=True) 
        
        dict_over_ind = find_intersections(out) 
        
        return dict_over_ind
    
def calculate_join(dict_over_ind, g1_reind, g2_reind):
        area_total = 0
        data_aggreg = []

        # note to self: make sure to import shapely Polygon
        for index1, crim in g1_reind.iterrows():
            try:
                index1 = crim.pp_index
                sals_found = dict_over_ind[index1]

                for sal in range(len(sals_found)):
                    pom = g2_reind[g2_reind.sal_index == sals_found[sal]]['geometry']        

                    #if pom.intersects(crim['geometry']).values[0]:
                    area_int = pom.intersection(crim['geometry']).area.values[0]
                    if area_int>0:
                        area_total += area_int 
                        area_crim = crim['geometry'].area

                        area_popu = pom.values[0].area

                        popu_count = g2_reind[g2_reind.sal_index == sals_found[sal]]['PPL_CNT'].values[0]
                        murd_count = crim['murd_cnt']
                        pol_province = crim['province']
                        popu_frac = (area_int / area_popu) * popu_count# fraction of the pop area contained inside the crim
                        #print(popu_frac)
                        extra_info_col_names = ['DC_NAME','MN_NAME','MP_NAME','PR_NAME','SP_NAME']
                        
                        extra_info_col_codes = ['MN_CODE','MP_CODE','PR_CODE','SAL_CODE','SP_CODE']

                        extra_names = g2_reind[g2_reind.sal_index == sals_found[sal]][extra_info_col_names]#.filter(regex=("NAME"))
                        extra_codes = g2_reind[g2_reind.sal_index == sals_found[sal]][extra_info_col_codes]#.filter(regex=("NAME"))

                        data_aggreg.append({'geometry': pom.intersection(crim['geometry']).values[0], 'id1': index1,\
                                      'id2': sals_found[sal] ,'area_pp': area_crim,'area_sal': area_popu,\
                                  'area_inter': area_int, 'popu_inter' : popu_frac, 'popu_sal': popu_count,\
                                  'murd_cnt': murd_count,'province': pol_province,
                                  'DC_NAME': extra_names.DC_NAME.values[0],\
                                  'MN_NAME': extra_names.MN_NAME.values[0], 'MP_NAME': extra_names.MP_NAME.values[0],\
                                  'PR_NAME': extra_names.PR_NAME.values[0],'SP_NAME': extra_names.SP_NAME.values[0],\
                                  'MN_CODE': extra_codes.MN_CODE.values[0],'MP_CODE': extra_codes.MP_CODE.values[0],\
                                  'PR_CODE': extra_codes.PR_CODE.values[0],'SAL_CODE': extra_codes.SAL_CODE.values[0],\
                                  'SP_CODE': extra_codes.SP_CODE.values[0]} )
            except:
                pass
            
        df_t = gpd.GeoDataFrame(data_aggreg,columns=['geometry', 'id1','id2','area_pp',\
                                       'area_sal','area_inter', 'popu_inter',\
                                       'popu_sal', 'murd_cnt','province','DC_NAME',\
                                       'MN_NAME','MP_NAME','PR_NAME','SP_NAME',\
                                      'MN_CODE','MP_CODE','PR_CODE','SAL_CODE','SP_CODE'])
        #df_t.to_file(out_name)
        return df_t, area_total, data_aggreg

In [43]:
# this function adds the remaining columns, calculates fractions etc
def compute_final_col(df_temp):
    # add population data per police percinct to the main table
    # id1- PP, id2 - SAL
    temp = df_temp.groupby(by=['id1'])['popu_inter'].sum().reset_index()

    data_with_population = pd.merge(df_temp, temp, on='id1', how='outer')\
            .rename(columns={'popu_inter_y':'popu_frac_per_pp', 'popu_inter_x':'popu_inter'})

    # finally, update the murder rate per SAL : id2 is sal's id    

    data_with_population['murd_est_per_int'] = data_with_population['popu_inter']/data_with_population['popu_frac_per_pp']\
               * data_with_population['murd_cnt']
    data_mur_per_int = data_with_population.groupby(by=['id2'])['murd_est_per_int'].sum().reset_index()

    data_mur_per_sal = data_mur_per_int.rename(columns={'murd_est_per_int':'murd_est_per_sal'})

    data_with_population['ratio_per_int'] = data_with_population['popu_inter']/data_with_population['popu_frac_per_pp']\

    data_complete = pd.merge(data_with_population, data_mur_per_sal, on='id2', how='outer')\
            .rename(columns={'id1':'index_PP', 'id2':'index_SAL'})
    return data_complete

# the geometry can be simplified using Line Simpl. algorithms
# data_with_pop['geo_simplified'] = data_with_pop.geometry.simplify(1000)
# for use one switches to a chosen simplified geometry:
# data_with_pop.set_geometry('geo_simplified', inplace=True)

Main functions to find intersection. Files loaded in are the AEA projected shapefiles.

In [4]:
salSHP_upd = 'shapefiles/updated/sal_population_aea.shp'
polSHP_upd = 'shapefiles/updated/polPrec_murd2015_prov_aea.shp'

geo_pol = gpd.GeoDataFrame.from_file(polSHP_upd)
geo_sal = gpd.GeoDataFrame.from_file(salSHP_upd)

geo_pol_reind = geo_pol.reset_index().rename(columns={'index':'pp_index'})
geo_sal_reind = geo_sal.reset_index().rename(columns={'index':'sal_index'})

dict_int = calculate_join_indices(geo_pol_reind,geo_sal_reind)
print("1")
dict_inc = calculate_inclusion_indices(geo_pol_reind, geo_sal_reind)

1
2
3


test on a subset:

In [ ]:
gt1= geo_pol_reind[geo_pol.province=="Free State"].head(n=2)
gt2 = geo_sal_reind[geo_sal_reind.PR_NAME=="Free State"].reset_index()
d = calculate_join_indices(gt1, gt2)

Running the intersections on pre-computed indices:

In [17]:
from timeit import default_timer as timer

start = timer() 

df_inc, sum_area_inc, data_inc = calculate_join(dict_inc, geo_pol_reind, geo_sal_reind)
end = timer()
print("1st", end - start)  

start = timer() 
df_int, sum_area_int, data_int = calculate_join(dict_int, geo_pol_reind, geo_sal_reind)
end = timer()
print("2nd", end - start)  

1st 1544.5270484819994
2nd 2133.577747626994


In [20]:
df_int.head(n=3)

,geometry,id1,id2,area_pp,area_sal,area_inter,popu_inter,popu_sal,murd_cnt,province,DC_NAME,MN_NAME,MP_NAME,PR_NAME,SP_NAME,MN_CODE,MP_CODE,PR_CODE,SAL_CODE,SP_CODE
0,(POLYGON ((179538.273068833 -3355106.778516249...,0,28532,55396371.27358,1.981387e+08,17779.878339,0.003230,36,25,Free State,Mangaung,Mangaung,Mangaung NU,Free State,Mangaung NU,499,499002,4,4990011,499002001
1,POLYGON ((181104.0571247448 -3347250.428848253...,0,29666,55396371.27358,7.408164e+06,7337878.480101,268.428873,271,25,Free State,Mangaung,Mangaung,Botshabelo,Free State,Botshabelo SP,499,499030,4,4990063,499030008
2,POLYGON ((176437.6520079001 -3347422.027408466...,0,29636,55396371.27358,2.894491e+05,289449.084550,707.000000,707,25,Free State,Mangaung,Mangaung,Botshabelo,Free State,Botshabelo G,499,499030,4,4991112,499030005


find pol precincts within WC boundary

In [ ]:
za_province = gpd.read_file('za-provinces.topojson',driver='GeoJSON')#.set_index('id')
za_province.crs={'init': '27700'}

wc_boundary = za_province.ix[8].geometry # WC
#pp_WC = geo_pol[geo_pol.geometry.within(wc_boundary)]
pp_WC_in = geo_pol[geo_pol.geometry.intersects(wc_boundary)]
#.unary_union, sal_wc_union_bound = sal_WC_in.unary_union
pp_WC_overlaps = pp_WC_in[pp_WC_in.province!="Western Cape"]
pp_WC_pol_annot = pp_WC_in[pp_WC_in.province=="Western Cape"]

In [ ]:
#pp_test = pp_WC_in[pp_WC_in['compnt_nm'].isin(['atlantis','philadelphia','kraaifontein','brackenfell','kuilsriver','kleinvleveerste river','macassar','somerset west','fish hoek'])]
#pp_test = pp_WC_in[pp_WC_in['compnt_nm'].isin(['beaufort west','doring bay','murraysburg', 'strandfontein','nuwerus','lutzville'])]
%matplotlib inline
pp_WC_overlaps.plot()

To compare, we load the data without the AEA projection:

In [ ]:
df_int_obs =gpd.GeoDataFrame.from_file('data/intersection_int_ext.shp')

In [ ]:
dfFS = df[df.PR_NAME=="Free State"]
dfoFS = df_int_obs[df_int2.PR_NAME=="Free State"]

In [ ]:
dfoFS[dfoFS.id1 == 0]['popu_frac']

In [ ]:
dfFS[dfFS.id1 == 0]['popu_frac'].sum()

Add the final columns and save the files +  check the sums across provinces.
First select only the SALs that are included at least 50% in the intersection

In [78]:
df_int.head(n=2)

,geometry,id1,id2,area_pp,area_sal,area_inter,popu_inter,popu_sal,murd_cnt,province,DC_NAME,MN_NAME,MP_NAME,PR_NAME,SP_NAME,MN_CODE,MP_CODE,PR_CODE,SAL_CODE,SP_CODE
0,(POLYGON ((179538.273068833 -3355106.778516249...,0,28532,55396371.27358,1.981387e+08,17779.878339,0.003230,36,25,Free State,Mangaung,Mangaung,Mangaung NU,Free State,Mangaung NU,499,499002,4,4990011,499002001
1,POLYGON ((181104.0571247448 -3347250.428848253...,0,29666,55396371.27358,7.408164e+06,7337878.480101,268.428873,271,25,Free State,Mangaung,Mangaung,Botshabelo,Free State,Botshabelo SP,499,499030,4,4990063,499030008


Additing additional columns and computing the lower/upper estimation bounds (selecting SALs overlapping at >50% or completely):

In [84]:
#list_mit =[]
list_int_inc = []
for i,entry in df_int.iterrows():#f_inc_aea:
    #if (entry.area_inter/entry.area_sal>=0.5): # select intersections where SAL overlap >50%
     #   list_mit.append(entry)
    if (entry.area_inter/entry.area_sal==1): # select those included 'completely'
        list_int_inc.append(entry)
df_int_mit=gpd.GeoDataFrame(list_mit)
df_int_inc=gpd.GeoDataFrame(list_int_inc)

In [82]:
df_int_aea_mit = compute_final_col(df_int_mit) # mitad= mid
df_int_aea_cpt = compute_final_col(df_int_inc) # complete

In [87]:
df_int_aea_mit.to_csv('data/pp_sal_aea_mit.csv')
df_int_aea_cpt.to_csv('data/pp_sal_aea_cpt.csv')

In [25]:
df_int_aea = compute_final_col(df_int) # all intersections
#df_inc_aea = compute_final_col(df_inc) # all full-inclusions
#df_inc_aea.to_csv('data/pp_sal_inc_included.csv')
df_int_aea.to_csv('data/pp_sal_inc_intersections.csv')

In [169]:
#df_inc_aea[df_int_aea.province != df_int_aea.MN_NAME][['province','MN_NAME']]

In [33]:
data_prov = df_int_aea[['PR_NAME','province','murd_est_per_int']]
data_prov.groupby('province')['murd_est_per_int'].sum()

province
0                   0
Eastern Cape     3051
Free State        943
Gauteng          3671
Kwazulu/Natal    3759
Limpopo           777
Mpumalanga        831
North West        853
Northern Cape     411
Western Cape     3186
Name: murd_est_per_int, dtype: float64

In [29]:
data_prov.groupby('PR_NAME')['murd_est_per_int'].sum()

PR_NAME
Eastern Cape     3051.126640
Free State        942.827525
Gauteng          3654.691802
KwaZulu-Natal    3758.865333
Limpopo           775.293714
Mpumalanga        832.509613
North West        872.349383
Northern Cape     408.343123
Western Cape     3185.992867
Name: murd_est_per_int, dtype: float64

In [31]:
# check over small areas- sum of all the crimes should be 17482
pom = {}
for ind, row in df_inc_aea.iterrows():
    pom[row['index_SAL']] = row['murd_est_per_sal'] 
s=0
for key in pom:
    s = s + pom[key]
print(s)

17482.00000000003


In [48]:
data_prov_ub = df_inc_aea[['PR_NAME','province','murd_est_per_int']]
data_prov_ub.groupby('province')['murd_est_per_int'].sum()

province
0                   0
Eastern Cape     3047
Free State        940
Gauteng          3670
Kwazulu/Natal    3758
Limpopo           777
Mpumalanga        830
North West        846
Northern Cape     410
Western Cape     3186
Name: murd_est_per_int, dtype: float64

In [35]:
data_prov = df_int_aea[['PR_NAME','province','murd_est_per_int']]
data_prov.groupby('province')['murd_est_per_int'].sum()

province
0                   0
Eastern Cape     3051
Free State        943
Gauteng          3671
Kwazulu/Natal    3759
Limpopo           777
Mpumalanga        831
North West        853
Northern Cape     411
Western Cape     3186
Name: murd_est_per_int, dtype: float64

Comparing bounds on the estimates for municipalities:

In [102]:
mn_names_set = set(df_int_aea_mit.MN_NAME)

In [106]:
mn_names = []
for s in mn_names_set:
    mn_names.append(s)

In [108]:
df_int_aea_mit.groupby(by=['MN_NAME'])['popu_inter'].sum()

'Abaqulusi'

In [143]:
df_int_aea[df_int_aea.SP_NAME=="Abashumi SP"] 

,geometry,index_PP,index_SAL,area_pp,area_sal,area_inter,popu_inter,popu_sal,murd_cnt,province,...,SP_NAME,MN_CODE,MP_CODE,PR_CODE,SAL_CODE,SP_CODE,popu_frac_per_pp,murd_est_per_int,ratio_per_int,murd_est_per_sal
9899,POLYGON ((654490.8727424829 -3328612.099414531...,80,38210,2.630121e+08,6533448.267764,6518911.568786,831.146603,833,14,Kwazulu/Natal,...,Abashumi SP,591,591003,5,5910160,591003001,43903.81752,0.265035,1.893108e-02,0.265928
9900,POLYGON ((652199.9817243671 -3326090.702572807...,339,38210,2.986221e+08,6533448.267764,14536.698978,1.853397,833,36,Kwazulu/Natal,...,Abashumi SP,591,591003,5,5910160,591003001,74737.33117,0.000893,2.479881e-05,0.265928
9901,"POLYGON ((653502.297831877 -3329189.587865365,...",80,38209,2.630121e+08,1435767.645138,1435394.023112,244.936245,245,14,Kwazulu/Natal,...,Abashumi SP,591,591003,5,5910009,591003001,43903.81752,0.078105,5.578928e-03,0.078136
9902,(POLYGON ((652263.1527263033 -3327343.22547291...,339,38209,2.986221e+08,1435767.645138,373.622026,0.063755,245,36,Kwazulu/Natal,...,Abashumi SP,591,591003,5,5910009,591003001,74737.33117,0.000031,8.530546e-07,0.078136
9903,POLYGON ((652320.9719487648 -3327581.795918106...,80,37803,2.630121e+08,597764.563633,597081.566295,132.848036,133,14,Kwazulu/Natal,...,Abashumi SP,589,589125,5,5890009,589125001,43903.81752,0.042362,3.025888e-03,0.042436
9904,(POLYGON ((651662.626213146 -3328579.482799806...,339,37803,2.986221e+08,597764.563633,682.997337,0.151964,133,36,Kwazulu/Natal,...,Abashumi SP,589,589125,5,5890009,589125001,74737.33117,0.000073,2.033307e-06,0.042436


In [ ]:
#f, ax = plt.subplots(1, figsize=(12, 12))
#wc.plot(ax=ax, color='grey', linewidth=0)
#gpd.plotting.plot_multipolygon(ax, sal_wc_union_bound, facecolor='green')

#gpd.plotting.plot_multipolygon(ax, sal_wc_union, facecolor='blue')

In [148]:
df_int_aea[df_int_aea_mit.SP_NAME=="!Kheis NU"] 

,geometry,index_PP,index_SAL,area_pp,area_sal,area_inter,popu_inter,popu_sal,murd_cnt,province,...,SP_NAME,MN_CODE,MP_CODE,PR_CODE,SAL_CODE,SP_CODE,popu_frac_per_pp,murd_est_per_int,ratio_per_int,murd_est_per_sal
46022,POLYGON ((-331093.3300993923 -3273366.49556798...,633,23927,1.749108e+10,6.124364e+07,4.308725e+07,289.154255,411,12,Northern Cape,...,!Kheis NU,380,380002,3,3800006,380002001,31979.871385,0.108501,0.009042,0.108501
46082,POLYGON ((-314299.534310644 -3274017.857864204...,636,23929,7.994457e+09,2.820613e+08,2.816493e+08,599.123535,600,8,Northern Cape,...,!Kheis NU,380,380002,3,3800010,380002001,15802.302056,0.303309,0.037914,0.303309
46083,POLYGON ((-305838.0587949275 -3286288.75652940...,636,23924,7.994457e+09,2.838504e+08,2.838504e+08,235.000000,235,8,Northern Cape,...,!Kheis NU,380,380002,3,3800001,380002001,15802.302056,0.118970,0.014871,0.118970
46084,POLYGON ((-305818.5324843627 -3286317.84787675...,636,23928,7.994457e+09,3.519636e+08,3.519636e+08,487.000000,487,8,Northern Cape,...,!Kheis NU,380,380002,3,3800008,380002001,15802.302056,0.246546,0.030818,0.246546
46085,POLYGON ((-329557.207027167 -3287754.453782321...,636,23930,7.994457e+09,6.106086e+08,6.106086e+08,1120.000000,1120,8,Northern Cape,...,!Kheis NU,380,380002,3,3800025,380002001,15802.302056,0.567006,0.070876,0.567006
51031,POLYGON ((-314138.9914293638 -3346346.34819530...,714,23926,1.948727e+10,5.936805e+09,5.662350e+09,369.109244,387,1,Northern Cape,...,!Kheis NU,380,380002,3,3800004,380002001,5723.839102,0.064486,0.064486,0.064486


In [152]:
df_int_aea[df_int_aea.SP_NAME=="700 Scheme"] 

,geometry,index_PP,index_SAL,area_pp,area_sal,area_inter,popu_inter,popu_sal,murd_cnt,province,...,SP_NAME,MN_CODE,MP_CODE,PR_CODE,SAL_CODE,SP_CODE,popu_frac_per_pp,murd_est_per_int,ratio_per_int,murd_est_per_sal
1284,POLYGON ((348671.0783432801 -3035094.885915838...,13,56507,1.446773e+07,142189.318919,142189.318919,501.000000,501,21,Gauteng,...,700 Scheme,797,797023,7,7971151,797023012,96029.802285,0.109560,0.005217,0.109560
1285,POLYGON ((348991.9001519837 -3034556.095969369...,13,56508,1.446773e+07,46369.843293,39740.105288,443.081817,517,21,Gauteng,...,700 Scheme,797,797023,7,7971279,797023012,96029.802285,0.096894,0.004614,0.141760
1286,POLYGON ((349003.2359490667 -3034536.085821455...,62,56508,3.192992e+08,46369.843293,6629.738005,73.918183,517,69,Gauteng,...,700 Scheme,797,797023,7,7971279,797023012,113680.864320,0.044866,0.000650,0.141760
1287,POLYGON ((348891.6647538714 -3034787.708902356...,13,56509,1.446773e+07,285077.114921,279770.703790,542.706486,553,21,Gauteng,...,700 Scheme,797,797023,7,7971571,797023012,96029.802285,0.118680,0.005651,0.124928
1288,POLYGON ((349302.6647362814 -3034942.380682283...,62,56509,3.192992e+08,285077.114921,5306.411130,10.293514,553,69,Gauteng,...,700 Scheme,797,797023,7,7971571,797023012,113680.864320,0.006248,0.000091,0.124928
1289,POLYGON ((348295.6426604269 -3035181.910826352...,13,56510,1.446773e+07,92150.641017,92150.641017,625.000000,625,21,Gauteng,...,700 Scheme,797,797023,7,7972180,797023012,96029.802285,0.136676,0.006508,0.136676
1290,POLYGON ((348230.6412798559 -3035024.928740057...,13,56511,1.446773e+07,116961.781022,116961.781022,663.000000,663,21,Gauteng,...,700 Scheme,797,797023,7,7972493,797023012,96029.802285,0.144986,0.006904,0.144986
1291,POLYGON ((348653.5141852071 -3034425.273493285...,13,56512,1.446773e+07,70962.763890,69911.575530,674.852931,685,21,Gauteng,...,700 Scheme,797,797023,7,7972678,797023012,96029.802285,0.147578,0.007028,0.153737
1292,POLYGON ((348660.9852435857 -3034412.451494046...,62,56512,3.192992e+08,70962.763890,1051.188360,10.147069,685,69,Gauteng,...,700 Scheme,797,797023,7,7972678,797023012,113680.864320,0.006159,0.000089,0.153737
1293,POLYGON ((349302.6647362814 -3034942.380682283...,13,56513,1.446773e+07,101450.516194,89951.705612,633.959017,715,21,Gauteng,...,700 Scheme,797,797023,7,7972858,797023012,96029.802285,0.138635,0.006602,0.187824


In [151]:
df_int_aea_cpt[df_int_aea_cpt.SP_NAME=="700 Scheme"] 

,geometry,index_PP,index_SAL,area_pp,area_sal,area_inter,popu_inter,popu_sal,murd_cnt,province,...,SP_NAME,MN_CODE,MP_CODE,PR_CODE,SAL_CODE,SP_CODE,popu_frac_per_pp,murd_est_per_int,ratio_per_int,murd_est_per_sal
617,POLYGON ((348671.0783432801 -3035094.885915838...,13,56507,14467727.202294,142189.318919,142189.318919,501,501,21,Gauteng,...,700 Scheme,797,797023,7,7971151,797023012,78405,0.134188,0.006390,0.134188
618,POLYGON ((348295.6426604269 -3035181.910826352...,13,56510,14467727.202294,92150.641017,92150.641017,625,625,21,Gauteng,...,700 Scheme,797,797023,7,7972180,797023012,78405,0.167400,0.007971,0.167400
619,POLYGON ((348230.6412798559 -3035024.928740057...,13,56511,14467727.202294,116961.781022,116961.781022,663,663,21,Gauteng,...,700 Scheme,797,797023,7,7972493,797023012,78405,0.177578,0.008456,0.177578
620,"POLYGON ((349172.99349525 -3035367.873377192, ...",13,56515,14467727.202294,106640.837774,106640.837774,929,929,21,Gauteng,...,700 Scheme,797,797023,7,7973866,797023012,78405,0.248823,0.011849,0.248823
621,"POLYGON ((349198.221173842 -3035485.567452256,...",13,56516,14467727.202294,91482.992508,91482.992508,1004,1004,21,Gauteng,...,700 Scheme,797,797023,7,7974040,797023012,78405,0.268911,0.012805,0.268911


In [158]:
temp_mit = df_int_aea_mit.groupby(by=['SAL_CODE'])['murd_est_per_int'].sum().reset_index().rename(columns={'murd_est_per_int': 'LB_murder'})

temp_est = df_int_aea.groupby(by=['SAL_CODE'])['murd_est_per_int'].sum().reset_index().rename(columns={'murd_est_per_int': 'LB_murder'})
# if we cound only complete SALs, there will be less people, thus higher crime rate per person
temp_cpt = df_int_aea_cpt.groupby(by=['SAL_CODE'])['murd_est_per_int'].sum().reset_index().rename(columns={'murd_est_per_int': 'UB_murder'})
# 232 municipalities
mn_bounds_t = pd.merge(temp_mit, temp_est, on='SAL_CODE', how='outer')\
            .rename(columns={'LB_murder_x':'LB', 'LB_murder_y':'est'})
mn_bounds = pd.merge(mn_bounds_t, temp_cpt, on='SAL_CODE', how='outer')

#       .rename(columns={'popu_inter_y':'popu_frac_per_pp', 'popu_inter_x':'popu_inter'})


In [160]:
temp_est.head()

,SAL_CODE,LB_murder
0,1600001,1.969173e+00
1,1600002,5.861464e-01
2,1600003,3.409414e-07
3,1600004,4.062001e-01
4,1600005,3.216966e-01


In [159]:
mn_bounds.head(n=20)

,SAL_CODE,LB,est,UB_murder
0,1600001,2.088889,1.969173e+00,NaN
1,1600002,0.591741,5.861464e-01,0.690684
2,1600003,0.000000,3.409414e-07,NaN
3,1600004,0.410077,4.062001e-01,0.478645
4,1600005,0.000000,3.216966e-01,NaN
5,1600006,0.379759,4.036340e-01,NaN
6,1600007,1.072511,1.010858e+00,NaN
7,1600008,0.558727,8.127318e-01,NaN
8,1600009,0.375564,3.720134e-01,0.438361
9,1600010,0.540960,5.118898e-01,0.731013


In [ ]:
f, ax = plt.subplots(1, figsize=(12, 12))
gpd.plotting.plot_multipolygon(ax, pp_WC_overlaps.unary_union, facecolor='red')
wc.plot(ax=ax, color='black', linewidth=0) 

WARDS:

In [ ]:
wardsShp =gpd.GeoDataFrame.from_file('../wards_delimitation/Wards_demarc/Wards2011.shp')

In [ ]:
wardsShp.head()

In [ ]:
wardsShp.crs